<a href="https://colab.research.google.com/github/hossein20s/Average_Blood_Pressure_estimation/blob/master/data_io_lib.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
Version = '3.4'
print('data_io_lib Version: ', Version)

In [0]:
import numpy as np

class data_reader():
    def __init__(self, filename, l=10, batchsize=32, random=True):
        df = None
        if(filename.endswith('.csv')):
          df = pd.read_csv(filename)
        elif(filename.endswith('.mat')):
          with h5py.File(filename, 'r') as f:
            print(list(f.keys()))
            tmp = f['Part_1']
            objectRef = tmp[0][0]
            ds = f[objectRef]
            dataframe = pd.DataFrame(index=index, columns=cols)
            print(len(cols))
            for i in range(len(cols)):
              print(cols[i])
              dataframe[cols[i]] = ds[:,i]
            df = dataframe
        # process the data into a matrix, and return the lenght
        print("Warning: Data passed should be normalized!")
        self.frac = 0.65
        self.random = random
        self.process(df, l)
        self.batchsize = batchsize

        self.pointer = 0
        self.epoch = 0
    def process(self, df, l):
        # Generate the data matrix
        df = df[["NBP (Mean)", "Minute Volume"]].dropna().as_matrix()
        length = df.shape[0]
        data = np.zeros((length-l, l))
        label = np.zeros((length-l, 1))
        for counter in range(length-l):
            data[counter, :] = df[counter: counter+l, 1]
            label[counter, :] = df[counter+l, 0]
        # Random shuffle
        length = data.shape[0]
        idx = np.random.choice(length, length, replace=False)
        if not self.random:
            idx = np.arange(length)
        self.val_idx = idx[int(self.frac*length):]

        shuf_data = data[idx, :]
        shuf_label = label[idx, :]
        self.data =data
        self.label = label

        self.train_data = shuf_data[:int(self.frac*length), :]
        self.train_label = shuf_label[:int(self.frac*length), :]
        self.train_size = int(self.frac*length)

        self.val_data = shuf_data[int(self.frac*length):, :]
        self.val_label = shuf_label[int(self.frac*length):, :]
        self.val_size = int((1-self.frac)*length)

        return None

    def get_next_train_batch(self):
        # getting the next train batch
        if self.pointer + self.batchsize >= self.train_size:
            end = self.train_size
            start = self.pointer
            self.pointer = 0
            self.epoch += 1
        else:
            end = self.pointer + self.batchsize
            start = self.pointer
            self.pointer += self.batchsize
        X = np.expand_dims(self.train_data[start:end, :], axis=-1)
        Y = self.train_label[start:end, :]
        return X, Y

    def get_val(self):
        X = np.expand_dims(self.val_data, axis=-1)

        return X, self.val_label[:]

    def get_whole(self):
        # get whole, for validation set
        X = np.expand_dims(self.data[:, :], axis=-1)
        Y = self.label[:, :]
        return X, Y

    def reset(self):
        self.pointer = 0
        self.epoch = 0

    def get_epoch(self):
        # return the current epoch
        return self.epoch

In [0]:
import pandas as pd

def normalize(df):
  #stats = dataframe.describe()
  return (df - df.mean()) / df.std()


In [0]:
import math
import numpy as np
import pandas as pd
import h5py
import sys

class DataLoader():
    """A class for loading and transforming data for the lstm model"""

    def __init__(self, filename, split, cols, index=None):
      if(filename.endswith('.csv')):
        dataframe = pd.read_csv(filename)
      elif(filename.endswith('.mat')):
        with h5py.File(filename, 'r') as f:
          print(list(f.keys()))
          tmp = f['Part_1']
          objectRef = tmp[0][0]
          ds = f[objectRef]
          dataframe = pd.DataFrame(index=index, columns=cols)
          print(len(cols))
          for i in range(len(cols)):
            print(cols[i])
            dataframe[cols[i]] = ds[:,i]
            
      print(dataframe.shape)
      dataframe = dataframe[:3000]
      print(dataframe.shape)
      print(dataframe.head(3))

      i_split = int(len(dataframe) * split)
      self.columns = cols
      self.data_train = dataframe.get(cols).values[:i_split]
      self.data_test  = dataframe.get(cols).values[i_split:]
      self.len_train  = len(self.data_train)
      self.len_test   = len(self.data_test)
      self.len_train_windows = None

    def get_train_data(self, seq_len, normalise, n_features):
      return self._get_train_test_data(seq_len, normalise, self.data_train, n_features)
    
    def get_test_data(self, seq_len, normalise, n_features):
      return self._get_train_test_data(seq_len, normalise, self.data_test, n_features)
    
    def _get_train_test_data(self, seq_len, normalise, data, n_features):
        '''
        Create x, y train data windows
        Warning: batch method, not generative, make sure you have enough memory to
        load data, otherwise use generate_training_window() method.
        '''
        data_x = []
        data_y = []
        for i in range(len(data) - 2*seq_len):
            x, y = self._next_2window(i, seq_len, normalise, data, n_features)
            data_x.append(x)
            data_y.append(y)
        return np.array(data_x), np.array(data_y)


    def _next_window(self, i, seq_len, normalise, data, n_features):
        '''Generates the next data window from the given index location i'''
        window = data[i:i+seq_len]
        normalized_window = self.normalise_windows(window, single_window=True)[0] if normalise else window
        x = normalized_window[:-1,:n_features]
        y = window[-1:, [1]]
        return x, y
      
    def _next_2window(self, i, seq_len, normalise, data, n_features):
        '''Generates the next data window from the given index location i'''
        window = data[i:i+2*seq_len]
        normalized_window = self.normalise_windows(window, single_window=True)[0] if normalise else window
        x = normalized_window[:seq_len,:n_features]
        y = window[seq_len:, [1]]
        return x, y
      

    def xxget_test_data(self, seq_len, normalise):
        '''
        Create x, y test data windows
        Warning: batch method, not generative, make sure you have enough memory to
        load data, otherwise reduce size of the training split.
        '''
        data_windows = []
        for i in range(self.len_test - seq_len):
            data_windows.append(self.data_test[i:i+seq_len])

        data_windows = np.array(data_windows).astype(float)
        data_windows = self.normalise_windows(data_windows, single_window=False) if normalise else data_windows

        x = data_windows[:, :-1]
        y = data_windows[:, -1, [0]]
        return x,y

    def get_base_train_data(self, seq_len, normalise):
        raw = pd.DataFrame( {  data.columns[0]: data.data_train[:,0] })
        x = normalize(raw)
        y = data.data_train[:,1]
        return x,y

    def get_base_test_data(self, seq_len, normalise):
        raw = pd.DataFrame( {  data.columns[0]: data.data_test[:,0] })
        x = normalize(raw)
        y = data.data_test[:,1]
        return x,y

    def seq2seq_train_data(self, seq_len, normalise):
        '''
        Create x, y train data windows
        Warning: batch method, not generative, make sure you have enough memory to
        load data, otherwise use generate_training_window() method.
        '''
        data_x = []
        data_y = []
        for i in range(self.len_train - seq_len):
            x, y = self._next_seq2seq_window(i, seq_len, normalise, self.data_train)
            data_x.append(x)
            data_y.append(y)
        return np.array(data_x), np.array(data_y)

    def generate_train_batch(self, seq_len, batch_size, normalise):
        '''Yield a generator of training data from filename on given list of cols split for train/test'''
        i = 0
        while i < (self.len_train - seq_len):
            x_batch = []
            y_batch = []
            for b in range(batch_size):
                if i >= (self.len_train - seq_len):
                    # stop-condition for a smaller final batch if data doesn't divide evenly
                    yield np.array(x_batch), np.array(y_batch)
                    i = 0
                x, y = self._next_window(i, seq_len, normalise)
                x_batch.append(x)
                y_batch.append(y)
                i += 1
            yield np.array(x_batch), np.array(y_batch)

    def get_normalised_column(self, column):
              x = column[0]
              if(x == 0):
                x = window[1]
                print(col_i, x)
              normalised_col = [((float(p) / float(x)) - 1) for p in column]
              return normalised_col

    def get_normalized_column(self, column):
      df = pd.DataFrame(column)
      normalized_col = normalize(df).values
      normalized_col = normalized_col.reshape(normalized_col.shape[0])
      return normalized_col
      

    def normalise_windows(self, window_data, single_window=False):
        '''Normalise window with a base value of zero'''
        normalised_data = []
        window_data = [window_data] if single_window else window_data
        for window in window_data:
            normalised_window = []
#            for col_i in range(window.shape[1]-1): # not normalizing the last column (label)
            for col_i in range(window.shape[1]): # use both columns as features
                normalised_col = self.get_normalized_column(window[:,col_i])
                normalised_window.append(normalised_col)
            normalised_window = np.array(normalised_window).T # reshape and transpose array back into original multidimensional format
            normalised_data.append(normalised_window)
        return np.array(normalised_data)


